In [234]:
# Import NumPy library
import numpy as np
import numpy as np
import cv2
import os
import time



In [235]:
n_subjects = 40
n_images = 8 # {5,6,7,8,9}


In [236]:
img_h = 112
img_w = 92
img_size = img_h * img_w
n_eigenfaces = 50 # Hyperparameter 

In [237]:
X_train = np.zeros((img_size, n_subjects * n_images))
z_train = np.zeros(n_subjects * n_images)

In [238]:
for i in range(n_subjects):
    for j in range(n_images):
        file_path = f'/kaggle/input/orl-data-set/datsets/s{i+1}/{j+1}.pgm'
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        img = np.array(img)
        X_train[:, i * n_images + j] = img.reshape(-1)
        z_train[i * n_images + j] = i + 1
    

In [239]:
# Record the start time
start_time = time.time()




# Compute the mean face vector and subtract it from each image vector
m = np.mean(X_train, axis=1)
A = X_train - m[:, None]

# Compute the covariance matrix of A and its eigenvectors and eigenvalues
C = A @ A.T
L, V = np.linalg.eig(A.T @ A)

# Sort the eigenvectors by their eigenvalues in descending order and select the top k eigenvectors
idx = np.argsort(L)[::-1]
V = V[:, idx[:n_eigenfaces]]

# Compute the eigenfaces by multiplying the eigenvectors of L with A
U = A @ V

# Project each image vector onto the eigenface space and get the feature vectors
Y = U.T @ A





In [240]:
def recognize_face(a):
    # Subtract the mean face vector from the image vector
    a = a - m
    # Project the image vector onto the eigenface space and get the feature vector
    y = U.T @ a
    # Compute the Euclidean distance between the feature vector and the training feature vectors
    dist = np.linalg.norm(Y - y[:, None], axis=0)
    # Find the index of the training image that has the smallest distance
    idx = np.argmin(dist)
    # Return the label of the predicted subject and the distance
    # The label is the subject ID of the closest match, which is stored in y
    return z_train[idx]

In [241]:

# Test the face recognition function on a new image
# Load the image and convert it to grayscale
labels=[]
z_test=[]
for i in range(n_subjects):
    for j in range(n_images+1,11,1):
        #print(j,f's{i+1}')
        test_img = cv2.imread(f'/kaggle/input/orl-data-set/datsets/s{i+1}/{j}.pgm', cv2.IMREAD_GRAYSCALE)
        test_img = np.array(test_img)
        z_test.append(i+1)
        #test_img = np.loadtxt(, dtype=np.uint8)
        # Reshape it to a vector
        test_img = test_img.reshape(-1)
        # Recognize the face and print the result
        #label, distance = recognize_face(test_img)
        labels.append(recognize_face(test_img))
        

In [242]:
arr=[]
for i in range(len(z_test)):
    if(z_test[i]-labels[i])==0:
        arr.append(1)
    else:
        arr.append(0)
np.sum(arr)/len(z_test)    

0.9375

In [243]:

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Time taken: {elapsed_time} seconds")


Time taken: 1.1684627532958984 seconds
